## 1. The main idea behind byte pair encoding (BPE)

- The main idea in BPE is to convert text into an integer representation (token IDs) for LLM training

![alt text](IMG/tokenization.jpg)

## 1.1 Bits and Bytes

- Before getting to the BPE algorithm, let's introduce the notion of bytes.
- Consider converting text into a btye array (BPE stands for "btye" pair encoding after all):

In [2]:
text="This is some text"
byte_arry=bytearray(text,"utf-8")
print(byte_arry)

bytearray(b'This is some text')


- When we call `list()` on a `bytearray` object,each byte is treated as an individual elements and the result is a list of integers corresponding to the byte values:

In [2]:
ids = list(byte_arry)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


- This would be a valid way to correct text into a token ID representation that we need for the embedding layer of an LLM.

- However,the downside of this approach is that is creating one ID for each character(that's a lot of IDs for a short text!)

- this means for a 17 character input text,we have to use 17 token IDs as input to the LLM:

In [4]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


- If you have worked with LLMs before you may know that BPE tokenizers have a vocabulary where we have a token IDs for whole words or subwords instead of each character.

- For example , the GPT-2 tokenizer tokenizes the small text("This is some text") into only instead of 17 tokens: `1212, 318,617,2420`

- You can double-check this using the interactive [tiktoken app](https://tiktokenizer.vercel.app/?model=gpt2) or the [tiktoken library:](https://github.com/openai/tiktoken)

![alt text](IMG/tiktokenizer.jpg)

In [3]:
import tiktoken

gpt2_tokenizer=tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode(text)

[1212, 318, 617, 2420]

- Since a byte consists of 8 bits, there are 28 = 256 possible values that a single byte can represent, ranging from 0 to 255

- You can confirm this by executing the code `bytearray(range(0, 257))`, which will warn you that` ValueError: byte must be in range(0, 256))`

- A BPE tokenizer usually uses these 256 values as its first 256 single-character tokens; one could visually check this by running the following code:


In [4]:
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")

0: !
1: "
2: #
3: $
4: %
5: &
6: '
7: (
8: )
9: *
10: +
11: ,
12: -
13: .
14: /
15: 0
16: 1
17: 2
18: 3
19: 4
20: 5
21: 6
22: 7
23: 8
24: 9
25: :
26: ;
27: <
28: =
29: >
30: ?
31: @
32: A
33: B
34: C
35: D
36: E
37: F
38: G
39: H
40: I
41: J
42: K
43: L
44: M
45: N
46: O
47: P
48: Q
49: R
50: S
51: T
52: U
53: V
54: W
55: X
56: Y
57: Z
58: [
59: \
60: ]
61: ^
62: _
63: `
64: a
65: b
66: c
67: d
68: e
69: f
70: g
71: h
72: i
73: j
74: k
75: l
76: m
77: n
78: o
79: p
80: q
81: r
82: s
83: t
84: u
85: v
86: w
87: x
88: y
89: z
90: {
91: |
92: }
93: ~
94: �
95: �
96: �
97: �
98: �
99: �
100: �
101: �
102: �
103: �
104: �
105: �
106: �
107: �
108: �
109: �
110: �
111: �
112: �
113: �
114: �
115: �
116: �
117: �
118: �
119: �
120: �
121: �
122: �
123: �
124: �
125: �
126: �
127: �
128: �
129: �
130: �
131: �
132: �
133: �
134: �
135: �
136: �
137: �
138: �
139: �
140: �
141: �
142: �
143: �
144: �
145: �
146: �
147: �
148: �
149: �
150: �
151: �
152: �
153: �
154: �
155: �
156: �
157: �
158:

- Above, note that entries 256 and 257 are not single-character values but double-character values (a whitespace + a letter), which is a little shortcoming of the original GPT-2 BPE Tokenizer (this has been improved in the GPT-4 tokenizer)

1.2 Building the vocabulary

- The goal of the BPE tokenization algorthim is to build a vocabulary of commonly occuring subwords like `298:ent` (which can be founded in entangle,enertain,entrance,entity,......,for example), or even complete words like 

318: is
617:some
1212: This
2420: text

1.3 BPE algorithm outline

**1. Identify frequent pairs**

- In each iteration,scan the text to find the most commomly occuring pair of bytes(or characters)

**2. Replace and record**

- Replace that pair with a new placeholder ID(one not already in use ,e.g if we start with 0...255, the first placeholder would be 256).

- Record this mapping in alookup table.
- The size of the lookup table is hyperparameter also called "vocab size(for GPT2 that's 50257)"

**3. Repeat until no gain**

- Keep repeating steps 1 and 2, continually merging the most frequent pairs

- Stop when no further compression is possible(e.g no pairs occurs more than once).

**Decompression(decoding)**

To restore the original text, reverse the process by substituting each ID with its corresponding pair, using the lookup table

## 1.4 BPE algorithm example

**1.4.1 Concrete example of the encoding part(steps 1 & 2 in section 1.3)**

- Suppose we have the text (training dataset) `the cat in the hat` from which we want to build the vocabulary for a BPE Tokenizer.

**Iteration 1**

`1. Identify the frequent pairs`

- In this text "th" appears twice (at the begiining and before the second "e")

`2. Replace and record`

- Replace "th" with new token ID that is not already in use ,e.g 256.
- The new text is: `<256>e cat in <256>e hat`
- The new vocabulary is 

        0:...

        ...

        256: "th"

**Iteration 2**

`1. Identify frequent pairs`

    - In the text `<256>e cat in <256>hat`, the pair `<256>e` appears twice 
`2. Replace and record`
    - Replace `<256e>` with new token ID that is not already in use, for example, `257.`

    -  The new text is:
        <257> cat in <257> hat

    - The updated vocabulary is:
        0:..

        ...

        256: "th"

        257: "<256>e"

**Iteration 3**

`1.Identify frequent pairs`

- In the text `<257> cat in <257> hat`, the pair `<257>`  appears twice (once at the beginning and once before “hat”).

`2.Replace and record`

- Replace <257>  with a new token ID that is not already in use, for example, 258.
- The new text is:
<258>cat in <258>hat
- The updated vocabulary is:
0: ...
...
256: "th"
257: "<256>e"
258: "<257> "
- and so forth


## 1.4.2 Concrete example of the decoding part (step 3 in section 1.3)

- To restore the original text, we reverse the process by substituting each token ID with its corresponding pair in the reverse order they were introduced
- Start with the final compressed text: `<258>cat in <258>hat`
- Substitute `<258> → <257> : <257> cat in <257> hat`
- Substitute `<257> → <256>e: <256>e cat in <256>e hat`
- Substitute `<256> → "th": the cat in the hat`


In [ ]:
from collections import Counter, deque
from functools import lru_cache
import json

class BPETokenizerSimple:
    def __init__(self):
        ## MAP TOKEN ID TO TOKEN_STR (e.g {11246:"some"})
        self.vocab={}
        ## MAP TOKEN_STR TO TOKEN_ID (e.g {"some": 11246})
        self.inverse_vocab={}
        ## DICTIONARY OF BPE MERGES: {(token_id1,token_id1): merged_token_id}
        self.bpe_merges={}
        ## FOR THE OFFICAL OPENAI GPT-2 MERGES , USES A RANK DICT:
        ## OF FORM {(string_A,string_B): rank}, WHERE LOWER RANK= HIGER PRIORITY
        self.bpe_ranks={}
    
    def train(self,text,vocab_size,allowed_special={"<|endoftext|>"}):
        """
        TRAIN THE BPE TOKENIZER FROM SCRATCH.

        Args:
            text (_type_): _description_
            vocab_size (_type_): _description_
            allowed_special (dict, optional): _description_. Defaults to {"<|endoftext|>"}.
        """

        ## PREPROCESS: REPLACE SPACES with "Ġ"
        ## NOTE THAT "Ġ" IS A PARTICULARY OF THE GPT-2 BPE IMPLEMENTATION
        ## E.g "Hello World" MIGHT BE TOKENIZED AS ["Hello","Ġworld"]
        ## (GPT-4 BPE would tokenize it as ['Hello','world'])
        processed_text=[]
        for i,char in enumerate(text):
            if char==" " and i!=0:
                processed_text.append("Ġ")
            if char !=" ":
                processed_text.append(char)
        
        
        ## INITIALIZE VOCAB WITH UNIQUE CHARACTERS INCLUDING   "Ġ" IF PRESENT
        ## START WITH FIRST 256 ASCII CHARACTERS

        unique_chars=[chr(i) for i in range(256)]
        unique_chars.extend(
            char for char in sorted(set(processed_text))
            if char not in unique_chars
            
        )

        if "Ġ" not in unique_chars:
            unique_chars.append("Ġ")
        
        self.vocab = {i: char for i,char in enumerate(unique_chars)}
        self.inverse_vocab = {char : i for i,char in self.vocab.items()}

        ## ADD ALLOWED SPECIAL TOKENS
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id=len(self.vocab)
                    self.vocab[new_id]=token
                    self.inverse_vocab[token]=new_id

        
        ## TOKENIZE THE PREPROCESSED TEXT INTO TOKEN IDs
        token_ids=[self.inverse_vocab[char] for char in processed_text]

        ## BPE STEP 1-3: Repeatedly find and replace frequent pairs
        for new_ids in range(len(self.vocab),vocab_size):
            pair_id=self.find_freq_pair(token_ids,mode="most")
            if pair_id==None:
                break
            token_ids=self.replace_pair(token_ids,pair_id,new_id)
            self.bpe_merges[pair_id]=new_ids
        ## BUILD THE VOCABULARY WITH MERGED TOKENS
        for (p0,p1),new_id in self.bpe_merges.items():
            mereged_token=self.vocab[new_id]=self.vocab[p0]+self.vocab[p1]
            self.vocab[new_id]=mereged_token
            self.inverse_vocab[mereged_token]=new_id
    def encode(self,text,allowed_special=None):
        """
        Encode the input text into a list of token IDs, with tiktoken-style handling of special tokens.
    
        Args:
            text (str): The input text to encode.
            allowed_special (set or None): Special tokens to allow passthrough. If None, special handling is disabled.
    
        Returns:
            List of token IDs.
        """
        import re
        token_ids=[]

        ## IF SPECIAL TOKEN HANDLING IS ENABLED
        if allowed_special is not None and len(allowed_special)>0:
            ## BUILD REGEX TO MATCH ALLOWED SPECIAL TOKENS
            special_pattern=(
                "("+"|".join(re.escape(tok) for tok in sorted(allowed_special,key=len,reverse=True)+ ")"
                             
                ))
            
    
    @staticmethod
    def find_freq_pair(token_ids,mode="most"):
        pairs=Counter(zip(token_ids,token_ids[1:]))

        if not pairs:
            return None
        if mode=="most":
            return max(pairs.items(),key=lambda x:x[1])[0]
        if mode=="least":
            return min(pairs.items(),key=lambda x:x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'")
    @staticmethod
    def replace_pair(token_ids,pair_id,new_id):
        dq=deque(token_ids)
        replaced=[]
        while dq:
            current=dq.popleft()
            if dq and (current,dq[0])==pair_id:
                replaced.append(new_id)
                ## Remove the 2nd token of the pair 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)
        return replaced
        

In [14]:
tokens_id=[100,102,104,1000,1000,1000]
pairs=Counter(zip(tokens_id,tokens_id[1:]))
pairs

Counter({(100, 102): 1, (102, 104): 1, (104, 1000): 1, (1000, 1000): 2})

In [19]:
max(pairs.items(),key=lambda x:x[1])

((1000, 1000), 2)

In [23]:
def find_freq_pair(token_ids,mode="most"):
        pairs=Counter(zip(token_ids,token_ids[1:]))

        if not pairs:
            return None
        if mode=="most":
            return max(pairs.items(),key=lambda x:x[1])[0]
        if mode=="least":
            return min(pairs.items(),key=lambda x:x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'")

In [42]:
def replace_pair(token_ids, pair_id, new_id):
    from collections import deque
    ## list-like container with fast appends and pops on either end
    dq = deque(token_ids)
    replaced = []
    while dq:
        ## Remove and return an element from the left side of the deque. If no elements are present, raises an IndexError.
        current = dq.popleft()
        
        if dq and (current, dq[0]) == pair_id:
            print("replacing",current,dq[0])
            replaced.append(new_id)
            dq.popleft()  # Remove second element of the pair
        else:
            replaced.append(current)
    return replaced


In [40]:
dq=deque(token_ids)
print(dq)
print(dq.popleft())
print(dq)

deque([1, 2, 3, 2, 3, 4, 5])
1
deque([2, 3, 2, 3, 4, 5])


In [45]:
# Input
token_ids = [1, 2, 70, 2, 3, 40, 50,40,50,40,50]
pair_id = find_freq_pair(token_ids)
print(pair_id)
new_id = 99

# Expected behavior:
# Finds every (2, 3) pair and replaces it with 99.
# So: [1, 2, 3, 2, 3, 4, 5] → [1, 99, 99, 4, 5]

output = replace_pair(token_ids, pair_id, new_id)
print(output)


(40, 50)
replacing 40 50
replacing 40 50
replacing 40 50
[1, 2, 70, 2, 3, 99, 99, 99]


In [27]:
allowed_special={"<|endoftext|>"}

In [28]:
def encode(text,allowed_special=None):
        """
        Encode the input text into a list of token IDs, with tiktoken-style handling of special tokens.
    
        Args:
            text (str): The input text to encode.
            allowed_special (set or None): Special tokens to allow passthrough. If None, special handling is disabled.
    
        Returns:
            List of token IDs.
        """
        import re
        token_ids=[]

        ## IF SPECIAL TOKEN HANDLING IS ENABLED
        if allowed_special is not None and len(allowed_special)>0:
            print(allowed_special)
            ## BUILD REGEX TO MATCH ALLOWED SPECIAL TOKENS
            special_pattern=(
                "("+"|".join(re.escape(tok) for tok in sorted(allowed_special,key=len,reverse=True)+ ")"
                             
                ))

        last_index=0
        token_ids=[]
        text="hello how are you?"
        for match in  re.finditer(special_pattern,text):
            prefix=text[last_index:match.start()]
            token_ids.append(encode(prefix,allowed_special=None))
            print(prefix,token_ids)
            
            
        

In [ ]:
def encode(text, allowed_special=None):
        """
        Encode the input text into a list of token IDs, with tiktoken-style handling of special tokens.
    
        Args:
            text (str): The input text to encode.
            allowed_special (set or None): Special tokens to allow passthrough. If None, special handling is disabled.
    
        Returns:
            List of token IDs.
        """
        import re
    
        token_ids = []
    
        # If special token handling is enabled
        if allowed_special is not None and len(allowed_special) > 0:
            # Build regex to match allowed special tokens
            special_pattern = (
                "(" + "|".join(re.escape(tok) for tok in sorted(allowed_special, key=len, reverse=True)) + ")"
            )
    
            last_index = 0
            for match in re.finditer(special_pattern, text):
                prefix = text[last_index:match.start()]
                token_ids.extend(encode(prefix, allowed_special=None))

                special_token=match.group(0)
                if special_token in inverse_vocab:
                     token_ids.append(inverse_vocab[special_token])
                else:
                     raise ValueError(f"Special token {special_token} not found in vocabulary.")
                last_index = match.end()
            
            text = text[last_index:]

            
                
                

In [47]:
input_text = "Jack embraced beauty through art and life.<|endoftext|> "

In [49]:
encode(input_text,allowed_special={'<|endoftext|>'})

TypeError: 'NoneType' object is not iterable